In [1]:
# %%capture
# import os
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth
# else:
#     # Do this only in Colab notebooks! Otherwise use pip install unsloth
#     !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
#     !pip install transformers==4.51.3
#     !pip install --no-deps unsloth

In [2]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install transformers==4.51.3
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.6.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from unsloth import FastLanguageModel
from transformers import TextStreamer
import json # 딕셔너리를 예쁘게 출력하기

/tmp/ipython-input-3-3802121506.py:3: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# 가장 간단한 Agent 형태

In [5]:
# Agent용 LLM모델로 "unsloth/Qwen3-14B" :

## LLM Model class

In [6]:
class LLMModel:
    def __init__(self, model_name: str = "unsloth/Qwen3-14B", max_seq_length: int = 2048,
                 load_in_4bit: bool = True, load_in_8bit: bool = False, full_finetuning: bool = False,
                 device_map: str = "auto"):
        # Initialize the LLMModel with model and tokenizer loaded internally.
        print("[LLMModel Init] 모델 및 토크나이저 로딩을 시작합니다...")
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            load_in_4bit=load_in_4bit,
            load_in_8bit=load_in_8bit,
            full_finetuning=full_finetuning,
            device_map=device_map,
        )
        print("[LLMModel Init] 모델 및 토크나이저 로딩 완료.")

    def generate_response(self, messages: list[dict], max_new_tokens: int = 1024, temperature: float = 0.6,
                         top_p: float = 0.95, top_k: int = 20) -> str:
        # Generate a response from the LLM based on the input messages.
        print("[LLMModel] 응답 생성 시작...")
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False ,  ## True
        )

        inputs = self.tokenizer(text, return_tensors="pt")
        model_device = next(self.model.parameters()).device
        inputs = {k: v.to(model_device) for k, v in inputs.items()}

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            streamer=TextStreamer(self.tokenizer, skip_prompt=True),
        )

        # 첫 번째 배치(=질문)에 대한 토큰 시퀀스를 가져와 디코딩
        decoded = self.tokenizer.decode(
            outputs[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        print("[LLMModel] 응답 생성 완료.")
        return decoded

## Agent class

In [7]:
class Agent:
    def __init__(self, llm_model: LLMModel):
        # 에이전트가 사용할 수 있는 외부 함수(도구)들을 여기에 등록합니다.
        self.tools = {}

        # TODO: Memory (메모리) 및 상태 관리 초기화
        self.memory = ""

        # 외부에서 주입된 LLMModel 설정
        self.llm_model = llm_model

        print("1. [Agent Init] 에이전트 초기화 완료.")

    def plan(self, query: str) -> list[str]:
        """
        [Planner] 사용자의 쿼리를 분석하고 실행 계획(단계)을 생성합니다.
        본 모듈에서는 키워드 기반의 간단한 라우팅(어떤 툴을 쓸지 결정)을 구현합니다.
        더 발전된 에이전트는 이 부분을 LLM을 이용해 동적으로 계획을 생성합니다.
        Args:     query (str): 사용자의 질문.

        Returns:  list[str]: 실행할 단계들의 리스트.
        """
        print(f"2. [Planner] 쿼리 분석 및 계획 수립 시작: '{query}'")
        # 현재는 매우 단순하게, 받은 쿼리 자체를 하나의 실행 단계로 간주합니다.
        # 과제 파트에서 이 부분을 더 지능적으로 수정할 예정입니다.
        steps = [query]
        print(f"3. [Planner] 계획 수립 완료: {steps}")
        return steps

    def execute(self, steps: list[str]) -> str:
        """
        [Executor] Planner가 생성한 계획에 따라 각 단계를 실행합니다.
        단계가 툴 호출인지, LLM에게 질문하는 것인지 판단하고 실행합니다.
        Args:   steps (list[str]): 실행할 단계들의 리스트.
        Returns:str: 최종 실행 결과.
        """
        print("4. [Executor] 계획 실행 시작...")
        # 현재는 오직 LLM 호출만 존재한다고 가정합니다.
        # steps 리스트의 모든 내용을 하나의 프롬프트로 합칩니다.
        prompt = "\n".join(steps)
        messages = [{"role": "user", "content": prompt}]

        # LLMModel을 통해 응답 생성
        result = self.llm_model.generate_response(messages)
        print("5. [Executor] 실행 완료.")
        return result

## LLMModel 인스턴스 생성

In [8]:
# LLMModel 인스턴스 생성
llm_model = LLMModel()

[LLMModel Init] 모델 및 토크나이저 로딩을 시작합니다...
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

[LLMModel Init] 모델 및 토크나이저 로딩 완료.


## Agent 생성 및 실행

In [9]:
# Agent 인스턴스 생성 (LLMModel 주입)
agent = Agent(llm_model)

# 사용자 쿼리 정의
query = "반도체 8대 공정 중에서 포토리소그래피(Photolithography) 단계에 대해서 자세히 설명해 줘."

# 에이전트 실행
# 계획 수립 (Plan)
steps = agent.plan(query)

# 계획 실행 (Execute)
answer = agent.execute(steps)

# 결과 출력
print("\n\n--- 최종 답변 ---")
print(answer)

1. [Agent Init] 에이전트 초기화 완료.
2. [Planner] 쿼리 분석 및 계획 수립 시작: '반도체 8대 공정 중에서 포토리소그래피(Photolithography) 단계에 대해서 자세히 설명해 줘.'
3. [Planner] 계획 수립 완료: ['반도체 8대 공정 중에서 포토리소그래피(Photolithography) 단계에 대해서 자세히 설명해 줘.']
4. [Executor] 계획 실행 시작...
[LLMModel] 응답 생성 시작...
반도체 제조 공정 중 **포토리소그래피**(Photolithography)는 반도체 소자 제작에 있어 가장 핵심적인 공정 중 하나로, **마이크로미터** 또는 **나노미터** 수준의 정밀한 회로 패턴을 반도체 기판 위에 전달하는 과정입니다. 이 공정은 반도체 8대 공정 중 **3번째 공정**으로, **패터닝**(Patterning) 공정에 해당합니다.

---

## 🔹 1. 포토리소그래피란?

**포토리소그래피**(Photolithography)는 **빛**(광선)을 이용해 **광감광막**(Photoresist)에 **패턴**을 인쇄하는 기술입니다. 이 패턴은 이후의 공정(예: 이온주입, 에칭 등)을 통해 반도체 기판에 **회로 구조**로 전달됩니다.

---

## 🔹 2. 포토리소그래피의 목적

- 반도체 기판 위에 **정밀한 회로 패턴**(예: 트랜지스터, 커패시터, 커넥터 등)을 만들기 위해
- **다음 공정**(이온주입, 에칭, 금속 증착 등)을 위한 **마스킹**(Masking)을 제공

---

## 🔹 3. 포토리소그래피의 주요 공정 단계

포토리소그래피는 다음과 같은 주요 단계로 구성됩니다:

---

### **1) 기판 준비 (Substrate Preparation)**

- **반도체 기판**(Silicon Wafer)에 **전도성 또는 절연성 층**(예: 산화막, 질화막 등)을 형성한 후
- **광감광막**(Photoresist)을 **도포**합니다.  
  - **광감광막**(Phot

# Agent 수정 : 정보 조회 추가

### 공정 파라미터를 조회하는 함수

In [10]:
# 더미(dummy) 공정 파라미터를 조회하는 함수

def get_step_parameters(step_name: str) -> dict:
    """
    주어진 공정 단계(step_name)의 파라미터를 조회하는 더미 함수입니다.
    실제 환경에서는 이 함수 내부에서 데이터베이스에 연결하여 데이터를 조회하는 로직이 들어갑니다.
    Args: step_name (str): 조회할 공정의 이름 (예: "포토리소그래피", "식각").
    Returns: dict: 해당 공정의 파라미터 딕셔너리. 없으면 빈 딕셔너리 반환.
    """
    print(f"--- [Tool Called] get_step_parameters(step_name='{step_name}') ---")

    # 실제 DB 대신 사용할 더미 데이터
    dummy_data = {
        "포토리소그래피": {"노광시간(Exposure Time)": "5.2초", "레진두께(PR Thickness)": "210nm", "초점(Focus)": "+0.1um"},
        "식각": {"가스(Gas)": "CF4 100sccm", "압력(Pressure)": "50mTorr", "전력(RF Power)": "300W"},
        "증착": {"온도(Temperature)": "450°C", "물질(Material)": "Si3N4", "두께(Thickness)": "50Å"}
    }
    # step_name 에 해당하는 키워드가 포함된 키를 찾아 반환
    for key, value in dummy_data.items():
        if key in step_name:
            return value
    return {}

In [ ]:
class Agent:
    def __init__(self, llm_model: LLMModel):
        # 툴 등록: 문자열 이름과 실제 함수를 매핑하는 딕셔너리
        self.tools = {
            "get_step_parameters": get_step_parameters
        }
        print(f"1. [Agent Init] 사용 가능한 툴: {list(self.tools.keys())}")

        # TODO: Memory (메모리) 및 상태 관리 초기화
        self.memory = ""

        # 외부에서 주입된 LLMModel 설정
        self.llm_model = llm_model

        print("1. [Agent Init] 에이전트 초기화 완료.")

    def plan(self, query: str) -> list[str]:
        print(f"2. [Planner] 쿼리 분석 및 계획 수립 시작: '{query}'")
        # 지능적인 계획 수립
        if "파라미터" in query or "정보" in query:
            # 공정 이름(예: '포토리소그래피')을 쿼리에서 추출
            step_name = query.split(" ")[0]  # 간단히 첫 단어를 공정 이름으로 가정
            plan = [f"TOOL_CALL:get_step_parameters({step_name})"]
        else:
            plan = [f"LLM_CALL:{query}"]

        print(f"3. [Planner] 계획 수립 완료: {plan}")
        return plan

    def execute(self, steps: list[str]) -> str:
        print("4. [Executor] 계획 실행 시작...")
        final_answer = ""
        for step in steps:
            if step.startswith("TOOL_CALL:"):
                # 툴 호출 명령 파싱
                tool_call_str = step.replace("TOOL_CALL:", "")
                tool_name = tool_call_str.split("(")[0]
                tool_arg = tool_call_str.split("(")[1][:-1]

                if tool_name in self.tools:
                    tool_function = self.tools[tool_name]
                    result = tool_function(tool_arg)
                    final_answer = json.dumps(result, indent=4, ensure_ascii=False)
                else:
                    final_answer = f"오류: '{tool_name}'은(는) 알 수 없는 도구입니다."
            elif step.startswith("LLM_CALL:"):
                # LLM 호출
                prompt = step.replace("LLM_CALL:", "")
                messages = [{"role": "user", "content": prompt}]
                final_answer = self.llm_model.generate_response(messages)

        print("5. [Executor] 실행 완료.")
        return final_answer


## Agent 생성 및 실행

In [11]:
# Agent 인스턴스 생성 (LLMModel 주입)
agent = Agent(llm_model)

# 사용자 쿼리 정의
query = "포토리소그래피 파라미터 정보 알려줘."

# 에이전트 실행
# 계획 수립 (Plan)
steps = agent.plan(query)

# 계획 실행 (Execute)
answer = agent.execute(steps)

# 결과 출력
print("\n\n--- 최종 답변 ---")
print(answer)

1. [Agent Init] 에이전트 초기화 완료.
2. [Planner] 쿼리 분석 및 계획 수립 시작: '포토리소그래피 파라미터 정보 알려줘.'
3. [Planner] 계획 수립 완료: ['포토리소그래피 파라미터 정보 알려줘.']
4. [Executor] 계획 실행 시작...
[LLMModel] 응답 생성 시작...
포토리소그래피(Photo-lithography)는 반도체, 미세전자기계시스템(MEMS), 폴리머 기반 소자 등에서 회로 패턴을 형성하기 위해 사용되는 핵심 공정입니다. 이 공정은 여러 단계로 이루어지며, 각 단계에는 다양한 파라미터들이 존재합니다. 아래는 포토리소그래피의 주요 단계와 해당 단계에서 사용되는 주요 파라미터들을 정리한 것입니다.

---

## 📌 포토리소그래피의 주요 단계 및 파라미터

### 1. **기판 준비 (Substrate Preparation)**
- **기판 재질:** 실리콘, 유리, 폴리머, 금속 등
- **표면 처리:** 청소, 화학적 층 제거, 표면 거칠기 조절
- **기판 두께:** 공정에 따라 50~500μm 등

---

### 2. **포토레지스트 도포 (Photoresist Coating)**
- **포토레지스트 종류:** 긍정형(Positive), 부정형(Negative)
- **도포 방법:** 스피닝(Spin Coating), 딥 코팅, 스프레이 코팅 등
- **도포 후 두께:** 0.1~5μm (일반적으로 1~2μm)
- **도포 조건:**
  - 스피닝 속도 (e.g., 1000~5000 RPM)
  - 도포 시간
  - 전도성 층 사용 여부 (전기적 방전 방지)

---

### 3. **전사 (Exposure)**
- **광원:** UV, DUV (Deep UV), E-Beam, X-ray 등
- **광원 파장:** 365nm, 248nm, 193nm 등
- **노광 에너지 (Exposure Dose):** 10~1000 mJ/cm²
- **노광 시간:** 1~60초 (광원 강도에 따라 달라짐